# The pyCERR SMIT HN CT Segmentation Model

## Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the OAR in head-and-neck region on a planning CT scan using pyCERR.

## Requirements
* Python>=3.8 (for pyCERR)
* Applying this model requires access to a A100-caliber Tensor Core GPU.

## AI model
* The segmentation model was trained and validated on CT scans used for RT planning. It does not work optimally on diagnostic CTs or scans in positions other than Head First Supine.
* The trained model is distributed along with python libraries and other dependencies via a conda package.

## Required input data
* RT planning CT DICOM

### Running the model

Update locations of input data and model directorues in section 2 of this notebook.
* Conda packge is location: `condaEnvDir`
* Inference script location: `wrapperPath`
* Inference script args


### License
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.
    
This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.

You may publish papers and books using results produced using software provided that you reference the appropriate citations:
*  SMIT model: https://arxiv.org/abs/2205.10342
*  CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
*  CERR software: https://doi.org/10.1118/1.1568978
*  CERR radiomics: https://doi.org/10.1002/mp.13046


YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.



In [ ]:
import os, subprocess
from glob import glob
import shutil
# work dir, set to your desired working directory
workDir = r'/home/user/hn_oar_smit_model'
os.makedirs(workDir,exist_ok=True)
os.chdir(workDir)

In [ ]:
%%capture
#!pip install pyxnat
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

In [ ]:
inputDicomPath = os.path.join(workDir,'input')
os.makedirs(inputDicomPath, exist_ok = True)

outputDicomPath = os.path.join(workDir, 'output')
os.makedirs(outputDicomPath, exist_ok = True)

sessionPath = os.path.join(workDir, 'session')
os.makedirs(sessionPath, exist_ok = True)

## Download planning CT DICOM data for processing

### Option 1: Download data from XNAT source (`getXNATData`)

In [ ]:
# # Function definition: pull data defined in scandict from XNAT
# from pyxnat import Interface
# import urllib3, shutil

# def getXNATData(xhost,user,scandict,downloadDir):
#   xnat = Interface(xhost, user, verify=False)
#   os.makedirs(downloadDir, exist_ok=True)
#   expdirlist = []
#   for scan_entry in scandict:
#     proj = scan_entry['proj']
#     subj = scan_entry['subj']
#     exp = scan_entry['exp']
#     scan_list = scan_entry['scan_list']
#     expdir = os.path.join(downloadDir,exp)
#     expdirlist.append(expdir)
#     os.makedirs(expdir, exist_ok = True)
#     xexp = xnat.select.project(proj).subject(subj).experiment(exp)
#     for scan in scan_list:
#       try:
#         xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
#       except:
#         xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
#     for dcmfolder in ['DICOM','secondary']:
#       dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
#       print(dcmlist)
#       for dcm in dcmlist:
#         shutil.move(dcm, expdir)
#   for dcmfolder in ['DICOM','secondary']:
#     if os.path.exists(os.path.join(downloadDir,dcmfolder)):
#       os.rmdir(os.path.join(downloadDir,dcmfolder))
#     if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
#       os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
#   xnat.disconnect()
#   return expdirlist

In [ ]:
# xhost = 'https://your.xnat'
# user = 'usr'
# scandict = [{'proj':'projectname','subj':'subject_ID','exp':'ID', 'scan_list':['1']}]

# dcmdirlist = getXNATData(xhost,user,scandict,inputDicomPath)

### Option 2:  Download data from other HTTP or BOX source

In [ ]:
# dataURL = 'http://datapath'
# dataDownloadDir = os.path.join(workDir,'tmp')
# os.makedirs(dataDownloadDir, exist_ok=True)
# os.chdir(dataDownloadDir)

# ! wget -O sampleData.gz -L {dataURL}
# ! tar xf sampleData.gz -C {inputDicomDir}
# ! rm sampleData.gz

# #unpack and move

In [ ]:
# dcmdirlist = glob(os.path.join(workDir,'input','*'))

### Option 3: Copy data from network share to `inputDicomPath`, or set `inputDicomPath` to location where your data resides.

In [ ]:
inputDicomPath = r'/path/to/dicom/dirs'
dcmdirlist = [os.path.join(inputDicomPath,x) for x in os.listdir(inputDicomPath) if x[:3] == 'HNC']

## Prepare to analyze data

In [ ]:
os.chdir(workDir)

In [ ]:
wrapperInstallDir = os.path.join(workDir,'CT_HN_SMIT')
condaEnvDir = os.path.join(wrapperInstallDir,'conda-pack')
condaEnvActivateScript = os.path.join(condaEnvDir, 'bin', 'activate')
wrapperPath = os.path.join(wrapperInstallDir,'bash_run_SMIT_Segmentation.sh')
model_basename = 'CT_HN_15_15_15'
load_weight_name = os.path.join(wrapperInstallDir,'trained_weights',model_basename + '.pt')

#pyCERR and filename parameters
scanNum = 0 # assume we are analyzing first scan in planC.scan/session data directory
scan_basename = 'CT'

In [ ]:
import subprocess
from cerr import plan_container as pc
from cerr.dcm_export import rtstruct_iod

In [ ]:
!git clone https://github.com/cerr/model_installer.git

In [ ]:
os.chdir(os.path.join(workDir,'model_installer'))

!./installer.sh -h

In [ ]:
%%capture
modelOpt = '7' #CT_HN_SMIT model
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

## Run the HN segmentation model

### Import DICOM with pycerr

In [ ]:
print(dcmdirlist)

In [ ]:
labels_dict = {1:'Brainstem',2:'Parotid_L',3:'Parotid_R',4:'Glnd_Submand_L',5:'Glnd_Submand_R',6:'Bone_Mandible',7:'Cavity_Oral',8:'SpinalCord',9:'Pterygoid_Lat_L',10:'Pterygoid_Lat_R',11:'Musc_Masseter_L',12:'Musc_Masseter_R',13:'Cartlg_Thyroid',14:'BrachialPlex_L',15:'BrachialPlex_R',16:'Larynx'}
structName = 'CT_HN_SMIT_plus'

In [ ]:
for dcmdir in dcmdirlist[:1]:
  subj = os.path.basename(dcmdir)
  sessiondir = os.path.join(sessionPath,subj)
  os.makedirs(sessiondir,exist_ok=True)

  #Convert input DICOM to NIfTI
  scan_niifile = os.path.join(sessiondir, scan_basename + '.nii')
  planC = pc.loadDcmDir(dcmDir = dcmdir)
  planC.scan[scanNum].saveNii(niiFileName = scan_niifile)

  numOrigStructs = len(planC.structure)

  #Run Segmentation
  os.chdir(wrapperInstallDir)
  cmd = f"source " + condaEnvActivateScript + " && source " + wrapperPath + " " + sessiondir + " " + sessiondir + " " + load_weight_name + " " + scan_niifile
  subprocess.run(cmd, shell=True, executable="/bin/bash")

  #Import output GTV NIfTI to pyCERR, and generate RTSTRUCT to match parent series
  output_niifile = os.path.join(sessiondir, model_basename + '_' + scan_basename + '.nii')
  planC = pc.loadNiiStructure(nii_file_name = output_niifile, assocScanNum = scanNum, planC = planC, labels_dict = labels_dict)
  numStructs = len(planC.structure)
  
  # Output segmentation to RTSTRUCT
  outputDir = os.path.join(workDir,'output')
  os.makedirs(outputDir, exist_ok=True)
  rtstructFile = os.path.join(outputDir, subj + '_RTSTRUCT.dcm')
  structNumV = range(numOrigStructs, numStructs) # Export the first and the last structure in the list
  seriesDescription = structName
  rtstruct_iod.create(structNumV = structNumV, filePath = rtstructFile, planC = planC, seriesOpts = {'SeriesDescription':seriesDescription})

  # remove session directory
  shutil.rmtree(sessiondir)

### Visualize

In [ ]:
from cerr.viewer import showMplNb
showMplNb(planC=planC, scan_nums=scanNum,
          struct_nums=structNumV, dose_nums=[],
          windowCenter=-400, windowWidth=2000)